<a href="https://colab.research.google.com/github/shaheen234/CryoET-Object-Identification/blob/main/CryoEt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files


In [ ]:


files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"shaheeenamir","key":"b1041c2c76006abac0330ef83725cb89"}'}

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c czii-cryo-et-object-identification

100% 8.45G/8.46G [01:58<00:00, 134MB/s]
100% 8.46G/8.46G [01:58<00:00, 76.6MB/s]


In [ ]:
# prompt: unzip data

!unzip czii-cryo-et-object-identification.zip


Archive:  czii-cryo-et-object-identification.zip
  inflating: sample_submission.csv   
  inflating: test/static/ExperimentRuns/TS_5_4/VoxelSpacing10.000/denoised.zarr/.zattrs  
  inflating: test/static/ExperimentRuns/TS_5_4/VoxelSpacing10.000/denoised.zarr/.zgroup  
  inflating: test/static/ExperimentRuns/TS_5_4/VoxelSpacing10.000/denoised.zarr/0/.zarray  
  inflating: test/static/ExperimentRuns/TS_5_4/VoxelSpacing10.000/denoised.zarr/0/0/0/0  
  inflating: test/static/ExperimentRuns/TS_5_4/VoxelSpacing10.000/denoised.zarr/0/0/0/1  
  inflating: test/static/ExperimentRuns/TS_5_4/VoxelSpacing10.000/denoised.zarr/0/0/0/2  
  inflating: test/static/ExperimentRuns/TS_5_4/VoxelSpacing10.000/denoised.zarr/0/0/1/0  
  inflating: test/static/ExperimentRuns/TS_5_4/VoxelSpacing10.000/denoised.zarr/0/0/1/1  
  inflating: test/static/ExperimentRuns/TS_5_4/VoxelSpacing10.000/denoised.zarr/0/0/1/2  
  inflating: test/static/ExperimentRuns/TS_5_4/VoxelSpacing10.000/denoised.zarr/0/0/2/0  
  inflating

In [ ]:
!pip install zarr

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 98.4 MB/s eta 0:00:00
  Created wheel for asciitree: filename=asciitree-0.3.3-py3-none-any.whl size=5034 sha256=beca5e86afd01e10ae68949b097210905be82bd255fd67cea625f876708e24db
  Stored in directory: /root/.cache/pip/wheels/7f/4e/be/1171b40f43b918087657ec57cf3b81fa1a2e027d8755baa184
Successfully built asciitree


In [ ]:
import os
import zarr
import json
import pandas as pd

# Paths to data files
static_zarr_files = [
    "train/static/ExperimentRuns/TS_5_4/VoxelSpacing10.000/denoised.zarr",
    "train/static/ExperimentRuns/TS_6_4/VoxelSpacing10.000/denoised.zarr",
    "train/static/ExperimentRuns/TS_69_2/VoxelSpacing10.000/denoised.zarr"
]
overlay_json_files = [
    "train/overlay/ExperimentRuns/TS_5_4/Picks/apo-ferritin.json",
    "train/overlay/ExperimentRuns/TS_6_4/Picks/ribosome.json",
    "train/overlay/ExperimentRuns/TS_69_2/Picks/beta-galactosidase.json"
]

# Initialize lists to store data
static_data = []
overlay_data = []

# Function to print memory usage
import psutil
import os

def print_memory_usage():
    process = psutil.Process(os.getpid())
    print(f"Memory Usage: {process.memory_info().rss / 1e9:.2f} GB")

# Read Zarr files from static paths
print("Processing static data...")
for zarr_file in static_zarr_files:
    experiment = os.path.basename(os.path.dirname(os.path.dirname(zarr_file)))
    if os.path.exists(zarr_file):
        z = zarr.open(zarr_file, mode='r')
        for index, chunk in enumerate(z['0']):
            static_data.append({'experiment': experiment, 'index': index, 'data': chunk.tolist()})
            print_memory_usage()  # Monitor memory usage during processing

# Save static data to a CSV file
static_df = pd.DataFrame(static_data)
static_df_path = 'static_chunk.csv'
static_df.to_csv(static_df_path, index=False)
print(f"Static data saved to {static_df_path}")

# Read JSON files from overlay paths
print("Processing overlay data...")
for json_file in overlay_json_files:
    experiment = os.path.basename(os.path.dirname(os.path.dirname(json_file)))
    if os.path.exists(json_file):
        with open(json_file, 'r') as f:
            overlay_content = json.load(f)
            for point in overlay_content['points']:
                overlay_data.append({
                    'experiment': experiment,
                    'particle_type': overlay_content['pickable_object_name'],
                    'x': point['location']['x'],
                    'y': point['location']['y'],
                    'z': point['location']['z']
                })

# Save overlay data to a CSV file
overlay_df = pd.DataFrame(overlay_data)
overlay_df_path = 'overlay_chunk.csv'
overlay_df.to_csv(overlay_df_path, index=False)
print(f"Overlay data saved to {overlay_df_path}")

# Load data back from CSV files for merging
print("Loading data for merging...")
static_df = pd.read_csv(static_df_path)
overlay_df = pd.read_csv(overlay_df_path)

# Merge DataFrames
print("Merging data...")
merged_df = pd.merge(overlay_df, static_df, on='experiment', how='inner')

# Save the merged DataFrame
merged_df_path = 'merged_data.csv'
merged_df.to_csv(merged_df_path, index=False)
print(f"Merged data saved to {merged_df_path}")

# Display a summary of the merged DataFrame
print("Merged DataFrame summary:")
print(merged_df.head())


Processing static data...
Processing overlay data...
Overlay data saved to overlay_chunk.csv
Loading data for merging...
Merging data...


OSError: [Errno 28] No space left on device

In [ ]:
import os
import zarr
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np


In [ ]:
# static_zarr_files = [
#     "train/static/ExperimentRuns/TS_5_4/VoxelSpacing10.000/denoised.zarr",
#     "train/static/ExperimentRuns/TS_6_4/VoxelSpacing10.000/denoised.zarr",
#     "train/static/ExperimentRuns/TS_69_2/VoxelSpacing10.000/denoised.zarr"
# ]
# overlay_json_files = [
#     "train/overlay/ExperimentRuns/TS_5_4/Picks/apo-ferritin.json",
#     "train/overlay/ExperimentRuns/TS_6_4/Picks/ribosome.json",
#     "train/overlay/ExperimentRuns/TS_69_2/Picks/beta-galactosidase.json"
# ]
# # Initialize lists to store data
# static_data = []
# overlay_data = []

# # Function to print memory usage
# import psutil
# import os

# def print_memory_usage():
#     process = psutil.Process(os.getpid())
#     print(f"Memory Usage: {process.memory_info().rss / 1e9:.2f} GB")

# # Read Zarr files from static paths
# print("Processing static data...")
# for zarr_file in static_zarr_files:
#     experiment = os.path.basename(os.path.dirname(os.path.dirname(zarr_file)))
#     if os.path.exists(zarr_file):
#         z = zarr.open(zarr_file, mode='r')
#         for index, chunk in enumerate(z['0']):
#             static_data.append({'experiment': experiment, 'index': index, 'data': np.array(chunk).flatten()})
#             print_memory_usage()  # Monitor memory usage during processing

# # Save static data to a DataFrame
# static_df = pd.DataFrame(static_data)

# # Read JSON files from overlay paths
# print("Processing overlay data...")
# for json_file in overlay_json_files:
#     experiment = os.path.basename(os.path.dirname(os.path.dirname(json_file)))
#     if os.path.exists(json_file):
#         with open(json_file, 'r') as f:
#             overlay_content = json.load(f)
#             for point in overlay_content['points']:
#                 overlay_data.append({
#                     'experiment': experiment,
#                     'particle_type': overlay_content['pickable_object_name'],
#                     'x': point['location']['x'],
#                     'y': point['location']['y'],
#                     'z': point['location']['z']
#                 })

# # Convert overlay data to a DataFrame
# overlay_df = pd.DataFrame(overlay_data)

# # Prepare features and labels

# print("Merging data...")
# merged_df = pd.merge(overlay_df, static_df, on='experiment', how='inner')



# # # Ensure the 'data' column is a proper NumPy array
# # merged_df['data'] = merged_df['data'].apply(lambda x: np.array(eval(str(x)), dtype=float))

Processing static data...
Memory Usage: 0.83 GB
Memory Usage: 0.83 GB
Memory Usage: 0.83 GB
Memory Usage: 0.83 GB
Memory Usage: 0.83 GB
Memory Usage: 0.83 GB
Memory Usage: 0.83 GB
Memory Usage: 0.84 GB
Memory Usage: 0.84 GB
Memory Usage: 0.84 GB
Memory Usage: 0.84 GB
Memory Usage: 0.55 GB
Memory Usage: 0.55 GB
Memory Usage: 0.55 GB
Memory Usage: 0.56 GB
Memory Usage: 0.56 GB
Memory Usage: 0.56 GB
Memory Usage: 0.56 GB
Memory Usage: 0.56 GB
Memory Usage: 0.56 GB
Memory Usage: 0.56 GB
Memory Usage: 0.57 GB
Memory Usage: 0.57 GB
Memory Usage: 0.57 GB
Memory Usage: 0.57 GB
Memory Usage: 0.57 GB
Memory Usage: 0.57 GB
Memory Usage: 0.58 GB
Memory Usage: 0.58 GB
Memory Usage: 0.58 GB
Memory Usage: 0.58 GB
Memory Usage: 0.58 GB
Memory Usage: 0.58 GB
Memory Usage: 0.59 GB
Memory Usage: 0.59 GB
Memory Usage: 0.59 GB
Memory Usage: 0.59 GB
Memory Usage: 0.59 GB
Memory Usage: 0.59 GB
Memory Usage: 0.59 GB
Memory Usage: 0.60 GB
Memory Usage: 0.60 GB
Memory Usage: 0.60 GB
Memory Usage: 0.60 GB
Memory

In [ ]:
# Ensure the 'data' column is a proper NumPy array
def convert_to_numpy_array(data):
    if isinstance(data, np.ndarray):
        return data  # Already a NumPy array
    elif isinstance(data, str):
        # Convert string representation back to NumPy array
        data = data.strip('[]').split(',')
        return np.array([float(i) for i in data])
    else:
        raise ValueError(f"Unexpected data format: {type(data)}")

# Apply conversion to all rows in 'data' column
merged_df['data'] = merged_df['data'].apply(convert_to_numpy_array)




In [ ]:
merged_df.to_csv('merged_data2.csv', index=False) # index=False prevents writing row indices to the file.

In [ ]:
# prompt: load merged_data_csv as dataframe

import pandas as pd

# Load the merged data from the CSV file
merged_data_df = pd.read_csv('merged_data2.csv')

# Display the first few rows of the DataFrame (optional)
print(merged_data_df.head())

  experiment particle_type        x         y        z  index  \
0     TS_5_4  apo-ferritin  468.514  5915.906  604.167      0   
1     TS_5_4  apo-ferritin  468.514  5915.906  604.167      1   
2     TS_5_4  apo-ferritin  468.514  5915.906  604.167      2   
3     TS_5_4  apo-ferritin  468.514  5915.906  604.167      3   
4     TS_5_4  apo-ferritin  468.514  5915.906  604.167      4   

                                                data  
0  [-1.0589503e-08  1.5891337e-06  8.6985551e-07 ...  
1  [-8.9148364e-07  1.9433514e-06  4.5870165e-06 ...  
2  [-2.1114688e-06  7.3233434e-07  3.9051638e-06 ...  
3  [-1.0797494e-06  1.3085246e-06  3.4091170e-06 ...  
4  [-6.0117713e-07  1.7272388e-06  2.2136107e-06 ...  


In [ ]:
# Ensure the 'data' column is a proper NumPy array
def convert_to_numpy(data):
    if isinstance(data, np.ndarray):
        return data  # Already a NumPy array
    elif isinstance(data, str):
        # Convert string representation of arrays to NumPy array
        data = data.strip('[]').split()
        return np.array([float(i) for i in data])
    else:
        raise ValueError(f"Unexpected data format: {type(data)}")

merged_df['data'] = merged_df['data'].apply(convert_to_numpy)


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
X = np.stack(merged_data_df['data'])  # Use flattened Zarr data as features
y = merged_data_df['particle_type']

In [ ]:
X[0]

'[-1.0589503e-08  1.5891337e-06  8.6985551e-07 ...  7.9891106e-07\n  4.1823807e-07  1.9056137e-07]'

In [ ]:
# prompt: convert numpy.str_ to numpy.array which is has comma seprated


# Remove rows with conversion errors (NaN values)
merged_data_df.dropna(subset=['data'], inplace=True)

# Check if the DataFrame is empty before stacking
if not merged_data_df.empty:
    X = np.stack(merged_data_df['data'].values)
    y = merged_data_df['particle_type']
else:
    print("Error: DataFrame is empty after conversion and NaN removal. Check your data.")
    X = None  # or handle the empty case appropriately
    y = None

In [ ]:
import numpy as np

# NumPy array of NumPy-like strings
data_strings = X

# Initialize a list to store the resulting NumPy arrays
numpy_arrays = []

# Iterate over the NumPy array of strings
for data_string in data_strings:
    # Remove square brackets and ellipsis
    cleaned_data = data_string.replace("[", "").replace("]", "").replace("...", "").strip()

    # Split by whitespace and convert to a comma-separated string
    elements = cleaned_data.split()
    comma_separated = ", ".join(elements)

    # Convert the comma-separated string to a NumPy array of floats
    numpy_array = np.array(comma_separated.split(", "), dtype=float)

    # Append to the list
    numpy_arrays.append(numpy_array)

# Convert the list of arrays into a NumPy array (optional, if needed)
numpy_arrays = np.array(numpy_arrays, dtype=object)  # dtype=object for ragged arrays




In [ ]:
numpy_arrays.shape

(25024, 6)

In [ ]:
y = y.astype('category').cat.codes

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(numpy_arrays, y, test_size=0.2, random_state=42)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# Scale the data for better neural network performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the model
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),  # First layer
    Dropout(0.3),  # Dropout for regularization
    Dense(128, activation='relu'),  # Second layer
    Dropout(0.3),
    Dense(64, activation='relu'),  # Third layer
    Dense(len(np.unique(y_train)), activation='softmax')  # Output layer
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# # Train the model
# print("Training neural network...")
# history = model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test),
#                     epochs=20, batch_size=32, verbose=1)

# # Evaluate the model
# print("\nEvaluating the model...")
# test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
# print(f"Test Accuracy: {test_accuracy:.2f}")

# # Generate predictions and classification report
# y_pred = model.predict(X_test_scaled)
# y_pred_labels = np.argmax(y_pred, axis=1)
# print("\nClassification Report:")
# print(classification_report(y_test, y_pred_labels))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 256)                 │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 43,139 (168.51 KB)

 Trainable params: 43,139 (168.51 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
X_train_scaled.shape


(20019, 6)

In [ ]:
history = model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test),
                    epochs=20, batch_size=32, verbose=1)

Epoch 1/20
626/626 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9654 - loss: 0.0981 - val_accuracy: 0.9970 - val_loss: 0.0213
Epoch 2/20
626/626 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9663 - loss: 0.0921 - val_accuracy: 0.9950 - val_loss: 0.0193
Epoch 3/20
626/626 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9695 - loss: 0.0887 - val_accuracy: 0.9964 - val_loss: 0.0181
Epoch 4/20
626/626 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9676 - loss: 0.0894 - val_accuracy: 0.9988 - val_loss: 0.0137
Epoch 5/20
626/626 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9693 - loss: 0.0821 - val_accuracy: 0.9982 - val_loss: 0.0117
Epoch 6/20
626/626 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9743 - loss: 0.0742 - val_accuracy: 0.9988 - val_loss: 0.0091
Epoch 7/20
626/626 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9757 - loss: 0.0701 - val_accuracy: 0.9982 - val_loss: 0.0098
Epoch 8/20
626/626 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9753 - loss: 0.0718 - val_accuracy: 0.

In [ ]:
model.save('model_competition.h5')

In [ ]:


# Prepare submission DataFrame
submission_data = []
test_root_dir = 'test/static/ExperimentRuns'

for experiment in os.listdir(test_root_dir):
    experiment_path = os.path.join(test_root_dir, experiment, 'VoxelSpacing10.000', 'denoised.zarr')
    if os.path.exists(experiment_path):

        print(f"Processing experiment: {experiment}")
        test_data = load_zarr_data(experiment_path)
        print(test_data)
            # # Scale test data (use the scaler fitted on training data)
            # scaler = StandardScaler()
            # X_test_scaled = scaler.fit_transform(X_test)
            # required_features = 6  # Replace with the number of features your model expects
            # if X_test.shape[1] != required_features:
            #     # Check if the feature size is smaller or larger and adjust accordingly
            #     if X_test.shape[1] < required_features:
            #         # Pad with zeros or duplicate columns to reach the required size
            #         padding = required_features - X_test.shape[1]
            #         X_test = np.hstack((X_test, np.zeros((X_test.shape[0], padding))))
            #     elif X_test.shape[1] > required_features:
            #         # Trim columns if there are too many features
            #         X_test = X_test[:, :required_features]

#             # Predict on test data



#             # Predict on test data
#             predictions = model.predict(X_test_scaled)

#             # Create submission data
#             for i, pred in enumerate(predictions):
#                 # Replace with actual coordinates if available
#                 submission_data.append({
#                     'experiment': experiment,
#                     'particle_type': pred,  # Predicted particle type
#                     'x': 0,  # Placeholder for x-coordinate
#                     'y': 0,  # Placeholder for y-coordinate
#                     'z': 0   # Placeholder for z-coordinate
#                 })
#     else:
#         print(f"Zarr file not found for experiment: {experiment}")

# # Convert to DataFrame
# submission_df = pd.DataFrame(submission_data)

# # Save to CSV
# submission_csv_path = 'submission.csv'
# submission_df.to_csv(submission_csv_path, index=False)
# print(f"Submission file saved to {submission_csv_path}")


Processing experiment: TS_6_4
['0' '1' '2']
Processing experiment: TS_5_4
['0' '1' '2']
Processing experiment: TS_69_2
['0' '1' '2']
